In [2]:
import pandas as pd
import numpy as np
import os
import sys
import json
import re
import time
import pathlib
from pathlib import Path
import janitor
from janitor import clean_names
import datetime

In [15]:
reporting_metrics_df = pd.read_excel(
    data_path, sheet_name="Information Source Characterist", skiprows=1
)

In [ ]:
# set current_dir to the directory of the notebook
current_dir = Path().resolve()  # This will get the current working directory
# load company total electricity use data
reporting_metrics_df = pd.read_excel(
    data_path, sheet_name="Information Source Characterist", skiprows=1
)
print(f"\nInitial company total records: {len(reporting_metrics_df)}")

data_path = current_dir.parent / "data" / "modules.xlsx"
columns_to_melt = [
    "Total company electricity use reporting?",
    "Total data center fleet electricity use reporting?",
    "Individual data center electricity use reporting?",
    "Data center fuel use reporting?",
    "PUE reporting?",
    "Total company water use reporting?",
    "Total data center fleet water  use reporting?",
    "Individual data center water use reporting?",
    "WUE reporting?",
    "Total company electric power sources reporting?",
    "Data center fleet electric power sources reporting?",
    "Individual data center electric power sources reporting?",
    "Third-party standards utilization?",
    "Total company Scope 1 GHG reporting?",
    "Data center fleet Scope 1 GHG reporting?",
    "Individual data center Scope 1 GHG reporting?",
    "Total company Scope 2 GHG reporting?",
    "Data center fleet Scope 2 GHG reporting?",
    "Individual data center Scope 2 GHG reporting?",
    "Total company Scope 3 GHG reporting?",
    "Data center fleet Scope 3 GHG reporting?",
    "Individual data center Scope 3 GHG reporting?",
]
columns_to_keep = [
    "Company",
    "Title",
    "Information source type",
    "File name",
    "Date released ",
    "Data year convention",
    "Fiscal year start",
    "Fiscal year end",
    "Geographical scope",
    "If partial, which locations are included?",
    "Source citation",
]
# Melt the columns into rows
reporting_metrics_df = pd.melt(
    reporting_metrics_df,
    id_vars=columns_to_keep,
    value_vars=columns_to_melt,
    var_name="metric",
    value_name="metric_value",
)
# extract the year from the file name value
reporting_metrics_df["report_year"] = reporting_metrics_df["File name"].str.extract(
    r"(?<!\d)(2\d{3})(?!\d)"
)
# extract the year from the title value if the report_year is NaN
title_years = reporting_metrics_df["Title"].str.extract(r"(?<!\d)(2\d{3})(?!\d)")[0]
# extract and flatten to Series
reporting_metrics_df["report_year"] = reporting_metrics_df["report_year"].fillna(
    title_years
)
# remove trailing spaces from the company name
reporting_metrics_df["Company"] = reporting_metrics_df["Company"].str.strip()

reporting_metrics_df["report_year"] = pd.to_numeric(
    reporting_metrics_df["report_year"], errors="coerce"
)

reporting_metrics_df.dropna(subset=["report_year"], inplace=True)
reporting_metrics_df["report_year"] = reporting_metrics_df["report_year"].astype(int)

# get the latest report year for each company
most_recent_years = (
    reporting_metrics_df.groupby("Company")["report_year"].max().reset_index()
)

# merge to get the latest metrics and values
recent_metrics_df = reporting_metrics_df.merge(
    most_recent_years, on=["Company", "report_year"]
)

export_path = current_dir.parent / "data" / "reporting_metrics_df.csv"
reporting_metrics_df.to_csv(export_path, index=False)
export_path = current_dir.parent / "data" / "recent_metrics_df.csv"
recent_metrics_df.to_csv(export_path, index=False)

reporting_metrics_df[reporting_metrics_df["Company"] == "Adobe"].head()



Initial company total records: 259


,Company,Title,Information source type,File name,Date released,Data year convention,Fiscal year start,Fiscal year end,Geographical scope,"If partial, which locations are included?",Source citation,metric,metric_value,report_year
193,Adobe,Adobe CDP Climate Change Questionnaire 2023,Annual Report,Adobe_Environmental_Report_CDP,7/27/ 2023,Fiscal,2022-12-03 00:00:00,2023-12-02 00:00:00,Full,NaN,Adobe. Adobe CDP Climate Change Questionnaire ...,Total company electricity use reporting?,N,2023
194,Adobe,Adobe - Climate Change 2022,Annual Report,Adobe_Environmental_Report_CDP_2022,NaN,Fiscal,2020-12-01 00:00:00,2021-11-30 00:00:00,Full,NaN,Adobe. Adobe - Climate Change 2022. 2023. http...,Total company electricity use reporting?,N,2022
195,Adobe,Adobe CDP Climate Change Questionnaire 2021,Annual Report,Adobe_Environmental_Report_CDP_2021,8/24/2020,Fiscal,2019-11-30 00:00:00,2020-11-29 00:00:00,Full,NaN,Adobe. Adobe CDP Climate Change Questionnaire ...,Total company electricity use reporting?,N,2021
196,Adobe,Adobe CDP Climate Change Questionnaire 2020,Annual Report,Adobe_Environmental_Report_CDP_2020,7/30/2021,Fiscal,2018-12-01 00:00:00,2019-11-30 00:00:00,Full,NaN,Adobe. Adobe CDP Climate Change Questionnaire ...,Total company electricity use reporting?,N,2020
197,Adobe,"Adobe, Inc. - Climate Change 2019",Annual Report,Adobe_Environmental_Report_CDP_2019,NaN,Fiscal,2018-12-01 00:00:00,2019-11-30 00:00:00,Full,NaN,"Adobe. Adobe, Inc. - Climate Change 2019. 2020...",Total company electricity use reporting?,N,2019
